In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '/Users/tareen/Desktop/Research_Projects/2022_mavenn2_github/mavenn')

import mavenn
import logomaker
import seaborn as sns
import re

%matplotlib inline

In [ ]:
# Load example data
full_wt_df = pd.read_csv('data/sortseq/full-wt/data.txt',delim_whitespace=True)
rnap_wt_df = pd.read_csv('data/sortseq/rnap-wt/data.txt',delim_whitespace=True)
crp_wt_df = pd.read_csv('data/sortseq/crp-wt/data.txt',delim_whitespace=True)

# # Separate test from data_df
# ix_test = data_df['set']=='test'
# test_df = data_df[ix_test].reset_index(drop=True)
# print(f'test N: {len(test_df):,}')

# # Remove test data from data_df
# data_df = data_df[~ix_test].reset_index(drop=True)
# print(f'training + validation N: {len(data_df):,}')


In [ ]:
data_df = full_wt_df.merge(rnap_wt_df,on='seq',how='outer').merge(crp_wt_df,on='seq',how='outer').fillna(0)

In [ ]:
y_cols = list(np.arange(0,30,1).astype(str))
cols = list(np.arange(0,30,1).astype(str))
cols.insert(0,'x')
data_df.columns = cols

In [ ]:
X = data_df['x']
y = data_df[y_cols]

In [ ]:
from sklearn.model_selection import train_test_split

# split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# from mavenn.src.layers.gpmap import GPMapLayer


# # Tensorflow imports
# import tensorflow as tf
# import tensorflow.keras.backend as K
# from tensorflow.keras.initializers import Constant
# from tensorflow.keras.layers import Layer, Dense

# class ThermodynamicLayer(GPMapLayer):
#     """Represents an thermodynamic model of transcription
#        regulation in E. Coli at the xylE promoter, which 
#        contains binding sites for RNAP, CRP, and xylR.
#     """

#     def __init__(self, 
#                  CRP_start,
#                  CRP_end,
#                  xylR_start,
#                  xylR_end,
#                  RNAP_start,
#                  RNAP_end,
#                  regularizer, 
#                  *args, **kwargs):
#         """Construct layer instance."""
        
#         # set attributes
#         self.CRP_start = CRP_start
#         self.CRP_end = CRP_end        

#         self.xylR_start = xylR_start
#         self.xylR_end = xylR_end                        
        
#         self.RNAP_start = RNAP_start
#         self.RNAP_end = RNAP_end        
        
#         #self.C=kwargs['C']
#         self.C=4
#         self.regularizer = tf.keras.regularizers.L2(regularizer)
        
#         # form helpful variables
#         self.L_CRP = CRP_end-CRP_start
#         self.L_RNAP = RNAP_end - RNAP_start
#         self.L_xylR = xylR_end - xylR_start
        
#         super().__init__(*args, **kwargs)

#     def build(self, input_shape):
#         """Build layer."""
        
#         #1 define bias/chemical potential weight for crp
#         self.mu_CRP = self.add_weight(name='mu_CRP',
#                                        shape=(1,),
#                                        initializer=Constant(0.),
#                                        trainable=True,
#                                        regularizer=self.regularizer)
        
        
#         #2 define bias/chemical potential weight for xylR
#         self.mu_xylR = self.add_weight(name='mu_xylR',
#                                        shape=(1,),
#                                        initializer=Constant(0.),
#                                        trainable=True,
#                                        regularizer=self.regularizer)                   

#         #3 define bias/chemical potential weight for rnap
#         self.mu_RNAP = self.add_weight(name='mu_RNAP',
#                                        shape=(1,),
#                                        initializer=Constant(0.),
#                                        trainable=True,
#                                        regularizer=self.regularizer)
     
        

#         #4 Define theta_CRP_lc parameters
#         theta_CRP_lc_shape = (1, self.L_CRP, self.C)

#         #theta_CRP_lc_init = np.random.randn(*theta_CRP_lc_shape)/np.sqrt(self.L_CRP)
#         self.theta_CRP_lc = self.add_weight(name='theta_CRP_lc',
#                                         shape=theta_CRP_lc_shape,
#                                         #initializer=Constant(CRP_init_values),
#                                         trainable=True,
#                                         regularizer=self.regularizer)

                
#         #5 Define theta_xylR_lc parameters
#         theta_xylR_lc_shape = (1, self.L_xylR, self.C)

#         #theta_xylR_lc_init = np.random.randn(*theta_xylR_lc_shape)/np.sqrt(self.L_xylR)
#         self.theta_xylR_lc = self.add_weight(name='theta_xylR_lc',
#                                         shape=theta_xylR_lc_shape,
#                                         #initializer=Constant(xylR_init_values),
#                                         trainable=True,
#                                         regularizer=self.regularizer)        
        
#         #6 Define theta_rnap_lc parameters
#         theta_RNAP_lc_shape = (1, self.L_RNAP, self.C)

#         #theta_RNAP_lc_init = np.random.randn(*theta_RNAP_lc_shape)/np.sqrt(self.L_RNAP)
#         self.theta_RNAP_lc = self.add_weight(name='theta_RNAP_lc',
#                                         shape=theta_RNAP_lc_shape,
#                                         #initializer=Constant(RNAP_init_values),
#                                         trainable=True,
#                                         regularizer=self.regularizer)

        
#         #7 define interaction term between CRP and xylR
#         self.I_cx = self.add_weight(name='I_cx',
#                                shape=(1,),
#                                initializer=Constant(-1),
#                                trainable=True)
        
#         #8 define interaction term between RNAP and xylR. 
#         self.I_rx = self.add_weight(name='I_rx',
#                                shape=(1,),
#                                initializer=Constant(-1),
#                                trainable=True,
#                                regularizer=self.regularizer)
        
# #         #9 define interaction term between CRP and RNAP
# #         self.I_cr = self.add_weight(name='I_cr',
# #                                shape=(1,),
# #                                initializer=Constant(-1),
# #                                trainable=True)        
        
# #         #10 define tsat term. 
#         self.tsat = self.add_weight(name='tsat',
#                                shape=(1,),
#                                initializer=Constant(1.),
#                                trainable=False)        
        
#         # Call superclass build
#         super().build(input_shape)

#     def call(self, x_lc):
#         """Process layer input and return output.
        
#         x_lc: (tensor)
#             Input tensor that represents one-hot encoded 
#             sequence values. 
#         """
        
#         # extract locations of binding sites from entire lac-promoter sequence.
#         x_CRP_lc = x_lc[:,self.C * self.CRP_start:self.C * self.CRP_end]
#         x_xylR_lc = x_lc[:,self.C * self.xylR_start: self.C * self.xylR_end]
#         x_RNAP_lc = x_lc[:,self.C * self.RNAP_start: self.C * self.RNAP_end]
        
#         # reshape according to crp and rnap lengths. 
#         x_CRP_lc = tf.reshape(x_CRP_lc, [-1, self.L_CRP, self.C])
#         x_xylR_lc = tf.reshape(x_xylR_lc, [-1, self.L_xylR, self.C])        
#         x_RNAP_lc = tf.reshape(x_RNAP_lc, [-1, self.L_RNAP, self.C])

                
#         # compute delta G for crp    
#         phi_CRP = self.mu_CRP + \
#               tf.reshape(K.sum(self.theta_CRP_lc * x_CRP_lc, axis=[1, 2]),
#                          shape=[-1, 1])
            
#         # compute delta G for LacI
#         phi_xylR = self.mu_xylR + \
#               tf.reshape(K.sum(self.theta_xylR_lc * x_xylR_lc, axis=[1, 2]),
#                          shape=[-1, 1])                             
            
#         # compute delta G for rnap
#         phi_RNAP = self.mu_RNAP + \
#               tf.reshape(K.sum(self.theta_RNAP_lc * x_RNAP_lc, axis=[1, 2]),
#                          shape=[-1, 1])                   
        
#         # partition function
#         Z = (1+K.exp(-phi_CRP)+K.exp(-phi_xylR)+ \
#              K.exp(-phi_RNAP)+ \
#              K.exp(-phi_xylR-phi_RNAP-self.I_rx)+ \
#              #K.exp(-phi_CRP-phi_RNAP-self.I_cr)+ \
#              K.exp(-phi_CRP-phi_RNAP)+ \
#              K.exp(-phi_CRP-phi_xylR-phi_RNAP-self.I_cx-self.I_rx) +\
#              K.exp(-phi_CRP-phi_xylR-self.I_cx)
#             )
        
#         transcription_states =  K.exp(-phi_RNAP)+ \
#                                 K.exp(-phi_xylR-phi_RNAP-self.I_rx)+ \
#                                 K.exp(-phi_CRP-phi_RNAP)+ \
#                                 K.exp(-phi_CRP-phi_xylR-phi_RNAP-self.I_cx-self.I_rx)
        
        
        
#         # compute rate of transcription
#         t = (self.tsat)*(transcription_states)/Z
        
        
#         # return rate of transcription
#         return t

In [ ]:
# set positional argumnets for gpmap function
L = len(data_df['x'][0])
alphabet=['A','C','G','T']

#y_cols = data_df.columns[1:-1]
Y = len(y_cols)
print(f'L={L}, Y={Y}')

# get test set data.

In [ ]:
# instantiate gpmap object
gpmap = mavenn.gpmap.AdditiveGPMapLayer(L, alphabet)

#therodynamic_gpmap = ThermodynamicLayer(alphabet=alphabet,**gpmap_kwargs)

In [ ]:
# I think this shouldn't require any positional arguments.
# mp_GE = mavenn.measurement_process_layers.GlobalEpistasisLayer(K=50,
#                                                                eta=1e-5,
#                                                                monotonic=True)
mp_mpa_1 = mavenn.measurement_process_layers.DiscreteAgnosticMP(info_for_layers_dict={'H_y_norm':0},
                                                                      Y=10,
                                                                      K=50,
                                                                      eta=1e-5,)

mp_mpa_2 = mavenn.measurement_process_layers.DiscreteAgnosticMP(info_for_layers_dict={'H_y_norm':0},
                                                                      Y=10,
                                                                      K=50,
                                                                      eta=1e-5,)

mp_mpa_3 = mavenn.measurement_process_layers.DiscreteAgnosticMP(info_for_layers_dict={'H_y_norm':0},
                                                                      Y=10,
                                                                      K=50,
                                                                      eta=1e-5,)


In [ ]:
model = mavenn.Model2(gpmap=gpmap,
                      mp_list=[mp_mpa_1, mp_mpa_2, mp_mpa_3])

In [ ]:
model.model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model.model,show_shapes=True,show_dtype=False,dpi=200)

In [ ]:

# Fit model to data
model.fit(learning_rate=.005,
          epochs=50,
          batch_size=1000,
          try_tqdm = False,
          early_stopping=True,
          early_stopping_patience=10,
          linear_initialization=False)

In [ ]:
# Show training history
print('On test data:')
# x_test = test_df['x'].values
# y_test = test_df[y_cols].values


I_var_hist = model.history['I_var']
val_I_var_hist = model.history['val_I_var']

fig, ax = plt.subplots(1,1,figsize=[4,4])
ax.plot(I_var_hist, label='I_var_train')
ax.plot(val_I_var_hist, label='I_var_val')
ax.legend()
ax.set_xlabel('epochs')
ax.set_ylabel('bits')
ax.set_title('training hisotry')

In [ ]:
phi = gpmap.x_to_phi(x_test)

In [ ]:
# Get G-P map parameters in matrix form
theta = np.squeeze(gpmap.weights[1].numpy())

# Create grid in phi space
#phi_lim = [-5, 3]
phi_lim = [min(phi), max(phi)]
phi_grid = np.linspace(phi_lim[0], phi_lim[1], 1000)

# Create array of allowable y values
Y = 10    # Y = number of bins
y_lim = [-.5, Y-.5]
y_all = range(Y)

# Compute matrix of p(y|phi) values
measurement_process_1 = mp_mpa_1.p_of_all_y_given_phi(phi).numpy()
measurement_process_2 = mp_mpa_2.p_of_all_y_given_phi(phi).numpy()
measurement_process_3 = mp_mpa_3.p_of_all_y_given_phi(phi).numpy()

measurement_process = np.hstack([measurement_process_1,measurement_process_2,measurement_process_3])

# Create figure with two panels
fig, axs = plt.subplots(1,2,figsize=[12,4])

# Left panel: draw logo using logomaker
ax = axs[0]
logo = logomaker.Logo(pd.DataFrame(theta,columns=model.alphabet), ax=ax,center_values=True)
ax.set_ylabel(r'parameter value ($\theta_{l:c}$)')
ax.set_xlabel(r'position ($l$)')
ax.set_title('G-P map parameters')

# Right panel: draw measurement process as heatmap
ax = axs[1]
im = ax.imshow(np.sort(measurement_process,axis=0).T,
               cmap='Greens',
               extent=phi_lim+y_lim,
               vmin=0,
               origin='lower',
               interpolation='nearest',
               aspect="auto")
ax.set_yticks(y_all)
ax.set_ylabel('bin number (y)')
ax.set_xlabel('latent phenotype ($\phi$)')
ax.set_title('measurement process')
cb = plt.colorbar(im)
cb.set_label('probability  $p(y|\phi)$', rotation=-90, va="bottom")

# Fix up plot
fig.tight_layout(w_pad=3)
#fig.savefig('sortseq_g-p_map_mpa_visualization.png')
plt.show()